### Esercitazione - E04 - Generative Adversarial Networks

Scarica un dataset di immagini reali, costruisci la tua GAN e genera nuove immagini!

* Usa il dataset [Oxford 102 Flowers](https://pytorch.org/vision/0.16/generated/torchvision.datasets.Flowers102.html#torchvision.datasets.Flowers102).

* Le immagini di questo dataset in genere hanno shape (3, 128, 128), ma per velocizzare il training puoi usare anche la dimensione (3, 64, 64).

* Costruisci una GAN basata su reti convoluzionali!

* Nei casi reali, le GAN sono molto sensibili al settaggio degli iperparametri e alla costruzione dell'architettura. Bisogna giocare un po' con gli iperparametri per essere sicuri che funzioni!

⚠️ Puoi utilizzare blocchi di codice che abbiamo scritto nei notebook precedenti!

In [1]:
# Import
import os

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
from torchvision.utils import save_image
from torchvision.datasets import Flowers102
import matplotlib.pyplot as plt

In [67]:
# Impostiamo gli hyperparametri
batch_size = 64
num_epoch = 100
z_dimension = 100  # dimensione del vettore di rumore in input al generatore

# Trasformazione delle immagini
def to_img(x):
    out = 0.5 * (x + 1)
    out = out.clamp(0, 1)
    out = out.view(-1, 3, 64, 64)
    return out

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((64, 64)),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [62]:
# Scarichiamo e prepariamo il dataset
dataset = Flowers102('./data', transform=img_transform, split="train", download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)

In [63]:
if not os.path.exists('/content/drive/MyDrive/RTD-A/Corsi/2023 - ProfessionAI/Colabs/dc_img'):
    os.mkdir('/content/drive/MyDrive/RTD-A/Corsi/2023 - ProfessionAI/Colabs/dc_img')


# Definiamo la classe dei modelli nella GAN:
# Discriminatore
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, 5, padding=2),  # batch, 32, 28, 28
            nn.LeakyReLU(0.2, True),
            nn.AvgPool2d(2, stride=2),  # batch, 32, 14, 14
            )
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, 5, padding=2),  # batch, 64, 14, 14
            nn.LeakyReLU(0.2, True),
            nn.AvgPool2d(2, stride=2)  # batch, 64, 7, 7
        )
        self.fc = nn.Sequential(
            nn.Linear(64*16*16, 1024),
            nn.LeakyReLU(0.2, True),
            nn.Linear(1024, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        '''
        x: batch, width, height, channel=1
        '''
        x = self.conv1(x)
        x = self.conv2(x)
        # print(x.shape)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Generatore
class Generator(nn.Module):
    def __init__(self, input_size, num_feature):
        super(Generator, self).__init__()
        self.fc = nn.Linear(input_size, num_feature)  # batch, 3136=1x56x56, 2352=3x28x28
        self.br = nn.Sequential(
            nn.BatchNorm2d(3),
            nn.ReLU(True)
        )
        self.downsample1 = nn.Sequential(
            nn.ConvTranspose2d(3, 50, kernel_size=5, stride=1),  # batch, 50, 56, 56
            nn.BatchNorm2d(50),
            nn.ReLU(True)
        )
        self.downsample2 = nn.Sequential(
            nn.ConvTranspose2d(50, 25, 2, stride=2),  # batch, 25, 56, 56
            nn.BatchNorm2d(25),
            nn.ReLU(True)
        )
        self.downsample3 = nn.Sequential(
            nn.ConvTranspose2d(25, 3, 1, stride=1),  # batch, 1, 28, 28
            nn.Tanh()
        )

    def forward(self, x):
        x = self.fc(x)
        x = x.view(x.size(0), 3, 28, 28)
        x = self.br(x)
        # print(x.shape)
        x = self.downsample1(x)
        # print("down1", x.shape)
        x = self.downsample2(x)
        # print("down2", x.shape)
        x = self.downsample3(x)
        # print("down3", x.shape)
        return x

In [74]:
# Creiamo un'istanza del Discriminatore
D = Discriminator().cuda()
# Creiamo un'istanza del Generatore
G = Generator(z_dimension, 2352).cuda()

# Definiamo la loss function
criterion = nn.BCELoss()

# Definiamo gli ottimizzatori per il discriminatore e il generatore
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0003)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.00003) #0.0003

In [ ]:
from tqdm import tqdm

# Alleniamo le nostre reti
for epoch in tqdm(range(num_epoch)):
    for i, (img, _) in enumerate(dataloader):
        num_img = img.size(0)
        # Alleniamo il Discriminatore
        real_img = img.cuda()
        real_label = torch.ones(num_img).cuda()
        fake_label = torch.zeros(num_img).cuda()

        # Calcoliamo la loss del Discriminatore sulle immagini reali
        real_out = D(real_img).squeeze(1)
        d_loss_real = criterion(real_out, real_label)
        real_scores = real_out  # Più è vicino a 1, meglio è

        # Calcoliamo la loss del Generatore sulle immagini generate
        z = torch.randn(num_img, z_dimension).cuda() # Noise di input
        fake_img = G(z)
        fake_out = D(fake_img).squeeze(1)
        d_loss_fake = criterion(fake_out, fake_label)
        fake_scores = fake_out  # Più è vicino a 0, meglio è

        # Ottimizzazione del Discriminatore
        d_loss = d_loss_real + d_loss_fake
        d_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()

        # Alleniamo il Generatore
        z = torch.randn(num_img, z_dimension).cuda()
        fake_img = G(z)
        output = D(fake_img).squeeze(1)
        # Calcoliamo la loss sulle immagini generate
        g_loss = criterion(output, real_label)

        # Ottimizzazione del Generatore
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()

        # if (i+1) % 100 == 0:
        #     print('Epoch [{}/{}], d_loss: {:.6f}, g_loss: {:.6f} '
        #           'D real: {:.6f}, D fake: {:.6f}'
        #           .format(epoch, num_epoch, d_loss.item(), g_loss.item(),
        #                   real_scores.data.mean(), fake_scores.data.mean()))
    if epoch == 0:
        real_images = to_img(real_img.cpu().data)
        save_image(real_images, '/content/drive/MyDrive/RTD-A/Corsi/2023 - ProfessionAI/Colabs/dc_img/real_images.png')

    fake_images = to_img(fake_img.cpu().data)
    save_image(fake_images, '/content/drive/MyDrive/RTD-A/Corsi/2023 - ProfessionAI/Colabs/dc_img/fake_images-{}.png'.format(epoch+1))

torch.save(G.state_dict(), '/content/drive/MyDrive/RTD-A/Corsi/2023 - ProfessionAI/Colabs/generator.pth')
torch.save(D.state_dict(), '/content/drive/MyDrive/RTD-A/Corsi/2023 - ProfessionAI/Colabs/discriminator.pth')

100%|██████████| 100/100 [13:00<00:00,  7.80s/it]


In [ ]:
# Generiamo nuovi campioni
z = torch.randn(4, z_dimension).cuda()
fake_img = G(z)
print(fake_img.shape)
fake_img = torch.permute(fake_img, (0, 2, 3, 1))

# Guardiamo i campioni generati
plt.subplot(2,2,1)
plt.imshow(fake_img.detach().cpu()[0])
plt.subplot(2,2,2)
plt.imshow(fake_img.detach().cpu()[1])
plt.subplot(2,2,3)
plt.imshow(fake_img.detach().cpu()[2])
plt.subplot(2,2,4)
plt.imshow(fake_img.detach().cpu()[3])

In [ ]:
fake_img.shape